In this code, we aim to use Edge Pooling and Unpooling in order to reconstruct the events. The main message passing technique so far is SAGE, and the main loss functions are MSE and Chamfer Distance 

# Libraries Installation


Import Pytorch Version 1.4 for compatibility with some libraries to be used like neural-net pytorch that contains EMD and Chamfer Distance Libraries

In [ ]:
!pip install -q torch==1.4.0 torchvision==0.5

In [ ]:

!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-geometric


In [ ]:
pip install "neuralnet-pytorch[geom] @ git+git://github.com/justanhduc/neuralnet-pytorch.git@master"

In [ ]:
!pip install neuralnet-pytorch

In [ ]:
!pip install git+git://github.com/justanhduc/neuralnet-pytorch.git@fancy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My\ Drive/gae/fast_gae/ECAL/temp

In [ ]:
import argparse
import time
import torch_geometric
import numpy as np
import scipy.sparse as sp
import torch
from torch import optim
from torch_geometric.data import Data

from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset, DataLoader
from itertools import cycle, islice
from torch_geometric.utils import dropout_adj


In [ ]:

from optimizer import loss_function
import iterableJet
from iterableJet import IterableMuons
import class_jet
from class_jet import FCMuonsGPU
from model import GraphPooling

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=200, help='Initial learning rate.') #100

parser.add_argument('--lr', type=float, default=0.0001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.4, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [ ]:
samples=torch.load('../ECAL Data /ECAL_f01_june_60k.pt')#
#samples=torch.load('../ECAL Data /ECAL_f01_june_60k.pt')
#samples=torch.load('../padded_June/newPad_0.pt')

In [ ]:
#samples=samples[:30000]

In [ ]:
for sam in samples:
  sam['x'][:,2]=-torch.log(sam['x'][:,2])*15
  #sam['x'][:,:2]/=150

In [ ]:

model = AliPooling(in_channels=3, out_channels1=64, out_channels2=128, out_channels3=256,out_channels4=512,out_channels5=1024,dropout=args.dropout,
                    batch_size=args.batch_size)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.001)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=0.001)

In [ ]:
def sizer(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

sizes=sizer(samples[:58000])

fin=[]
countit=0
for m in sizes:
    fin.append(np.repeat(countit,m))
    countit+=1


In [ ]:

from torch_geometric.data import DataLoader
#jet=IterableMuons(samples)
loader = DataLoader(samples[:58000], batch_size=args.batch_size,num_workers=2,pin_memory=True)

#%%
#model.train()
#model= model.to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.001)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0005, weight_decay=0.001)
#model.train()
#model.to(device)


In [ ]:
scheduler=StepLR(optimizer, step_size=4, gamma=0.1)

In [ ]:
checkpoint = torch.load('edge_mse_august.pth')
model.load_state_dict(checkpoint['model_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
model.eval()
model.to(device)

AliPooling(
  (sage1): SAGEConv(3, 64)
  (sage2): SAGEConv(64, 128)
  (sage3): SAGEConv(128, 256)
  (sage4): SAGEConv(256, 512)
  (tr1): Linear(in_features=512, out_features=1024, bias=True)
  (tr2): Linear(in_features=1024, out_features=2048, bias=True)
  (bano1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec0): Linear(in_features=2048, out_features=1024, bias=True)
  (dec1): Linear(in_features=1024, out_features=512, bias=True)
  (dec2): SAGEConv(512, 256)
  (dec3): SAGEConv(256, 128)
  (dec4): SAGEConv(128, 64)
  (dec5): SAGEConv(64, 3)
  (edge1): EdgePooling(64)
  (edge2): EdgePo

In [ ]:
testing = samples[-1000:]

sizes2=sizer(testing)


In [ ]:

fin2=[]
count23=0
for m2 in sizes2:
    fin2.append(np.repeat(count23,m2))
    count23+=1

from torch_geometric.data import DataLoader
Testloader = DataLoader(testing, batch_size=100)

In [ ]:
epoch2=checkpoint['epoch']

In [ ]:
for epoch in range(epoch2,50):
    #model.train()
    
    count=0
    c1,c2=0,args.batch_size
    epLoss=0
    t = time.time()
    for el in islice(loader,0,290):#count2,el in enumerate(loader):
        
        gra=el.x
        adj=el.edge_index



        check=torch.LongTensor(np.hstack(np.array(fin[c1:c2])))
        count+=1#=count2+1
        hidden_emb = None
        
        gra=gra.to(device)
        adj=adj.to(device)
        lengs=check.to(device)
        optimizer.zero_grad()
       

        r1 ,mu,sig= model(gra,adj,lengs,gra.shape[0])


        loss = loss_function(r1,gra,lengs,mu,sig) #+loss2(logsoftmax(torch.transpose(r2,1,2)),temp.long())      

        loss.backward()

        optimizer.step()

        

        cur_loss = loss.item()

        
        torch.cuda.empty_cache() 
        
        epLoss+=float(cur_loss)

        #hidden_emb = mu.data.numpy() 


        c1+=args.batch_size
        c2+=args.batch_size


        
        
        if count%145==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()

            with torch.no_grad():
              model.eval()
              VaLoss=0
              d1,d2=0,100
              for el2 in islice(Testloader,10):
                gra2=el2.x.cuda()  ##features 
                adj2=el2.edge_index.cuda() ## edge matrix 
                lengs2=torch.LongTensor(np.hstack(np.array(fin2[d1:d2]))).cuda()
                r12,mu2,sig2= model(gra2,adj2,lengs2,gra2.shape[0]) ## X , A 
                loss2 = loss_function(r12,gra2,lengs2,mu2,sig2)
                cur_loss2 = loss2.item()
                VaLoss+=float(cur_loss2)
                d1+=100
                d2+=100
              print("Val Loss:", '%04d' % (VaLoss/10))

    if epoch%2==0:
      scheduler.step()
      #cumLoss=validate(Testloader,5)
      torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch':epoch,
            'loss': loss,
            'epLoss':epLoss
            }, './edge_mse_august.pth')

           

Epoch: 0031 train_loss= 2413.71838 time= 1525.71009
Val Loss: 1389
Epoch: 0031 train_loss= 1887.91755 time= 1581.23402
Val Loss: 1384
Epoch: 0032 train_loss= 1364.61513 time= 1548.64045
Val Loss: 1381
Epoch: 0032 train_loss= 1359.91757 time= 1581.25350
Val Loss: 1378
Epoch: 0033 train_loss= 1359.44634 time= 1554.17089
Val Loss: 1376
Epoch: 0033 train_loss= 1355.17816 time= 1592.36344
Val Loss: 1374
Epoch: 0034 train_loss= 1355.62953 time= 1556.70469
Val Loss: 1373
Epoch: 0034 train_loss= 1351.53247 time= 1595.11426
Val Loss: 1371
Epoch: 0035 train_loss= 1352.39346 time= 1570.22319
Val Loss: 1369
Epoch: 0035 train_loss= 1348.38238 time= 1607.90768
Val Loss: 1368
Epoch: 0036 train_loss= 1350.07814 time= 1563.25482
Val Loss: 1368
Epoch: 0036 train_loss= 1346.74993 time= 1595.91331
Val Loss: 1368
Epoch: 0037 train_loss= 1349.80027 time= 1561.40026
Val Loss: 1367
Epoch: 0037 train_loss= 1346.47951 time= 1478.75834
Val Loss: 1367
Epoch: 0038 train_loss= 1349.54369 time= 1459.96496
Val Loss: 

In [ ]:
count

137

In [ ]:

#%%
testing = samples[-330:]

#%%
sizes2=sizer(testing)


In [ ]:
            
posMom=[]

            with torch.no_grad():
              model.eval()
              for el2 in islice(Testloader,1):
                gra2=el2.x.cuda()  ##features 
                adj2=el2.edge_index.cuda() ## edge matrix 
                lengs2=torch.LongTensor(np.hstack(np.array(fin2[:100]))).cuda()
                r12,mu2,sig2= model(gra2,adj2,lengs2,gra2.shape[0]) ## X , A 
                loss2 = loss_function(r12,gra2,lengs2,mu2,sig2)
                cur_loss2 = loss2.item()
                posMom.append(r12)
                print("Val Loss:", '%04d' % (cur_loss2))

Val Loss: 1384


In [ ]:

#%%
alles=[]
temp=0
for siz in sizes2:
  temp+=siz
  alles.append(temp)

In [ ]:
 %matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.colors import LogNorm

Using matplotlib backend: agg


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
real=testing[1]['x']

In [ ]:
%matplotlib inline
#xmin, xmax = min(lis4[:,0]), max(lis4[:,0])
#ymin, ymax = min(lis4[:,1]), max(lis4[:,1])
for k in range(10):
  real=testing[k]['x']
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-real[:,2]/10)
  plt.figure(figsize=(7,6))
  sc = plt.scatter(real[:,0],real[:,1],c=weights,cmap='viridis',norm=LogNorm(), alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()
  #plt.savefig('./Real2/fig'+str(k)+'.png')


In [ ]:
%matplotlib inline
#xmin, xmax = min(lis4[:,0]), max(lis4[:,0])
#ymin, ymax = min(lis4[:,1]), max(lis4[:,1])
k1,k2=0,1
for j in range(10):
  
  lis4=r12[alles[k1]:alles[k2]].detach().cpu().clone()
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-lis4[:,2]/10)
  plt.figure(figsize=(7,6))
  sc = plt.scatter(lis4[:,0],lis4[:,1],c=weights,cmap='viridis', norm=LogNorm(),alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()
  k1+=1
  k2+=1
  #plt.savefig('./Rec2/rec'+str(j)+'.png')


In [ ]:
  lis4=r12[215:691].detach().cpu().clone()
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-10*lis4[:,2])
  plt.figure(figsize=(7,6))
  sc = plt.scatter(lis4[:,0]*100,lis4[:,1]*100,c=weights,cmap='viridis', norm=LogNorm(),alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()

In [ ]:
%matplotlib inline
#xmin, xmax = min(lis4[:,0]), max(lis4[:,0])
#ymin, ymax = min(lis4[:,1]), max(lis4[:,1])

k1,k2=0,1
for j in range(9):
  lis4=r12[alles[k1]:alles[k2]].detach().cpu().clone()
  xmin,xmax=0,125
  ymin,ymax=0,125
  binsxy = [int((xmax - xmin) / 50), int((ymax - ymin) / 50)]
  weights=np.exp(-10*lis4[:,2])
  plt.figure(figsize=(7,6))
  sc = plt.scatter(lis4[:,0]*100,lis4[:,1]*100,c=weights,cmap='viridis', norm=LogNorm(),alpha=0.9)
  plt.colorbar(sc)
  check=np.arange(0,140,10)
  plt.xticks(check)
  plt.yticks(check)
  plt.grid()
  k1+=1
  k2+=1
  
  #plt.savefig('./Rec/rec'+str(j)+'.png')
